In [1]:
import numpy as np
import math
from scipy.stats import norm

## Asian options pricing

In [2]:
def pricer_mm(S0, K, r, sigma, T, n, option_type):
    
    M1 = S0 /n*sum([np.exp(r*(i+1)/n) for i in range(n)])
    print(M1)
    M2 = S0**2/n**2 * sum([np.exp(r*(i+j+2)/n + (sigma**2)*min(i+1,j+1)/n) for i in range(n) for j in range(n)])
    print(M2)
    param1 = 2*np.log(M1/S0) - np.log(M2/S0**2)/2
    param2 = math.sqrt(np.log(M2/S0**2) - 2*np.log(M1/S0))
    print([param1, param2])
    d1 = (np.log(S0/K) + param1 + param2**2) /param2
    d2 = d1 - param2
    if option_type == 'call':
        price = np.exp(-r*T) * (S0*np.exp(param1+(param2**2)/2)*norm.cdf(d1) - K*norm.cdf(d2) )
        print(np.exp(-r*T) )
        print(np.exp(param1+(param2**2)/2)*norm.cdf(d1))
        print(K*norm.cdf(d2))
    elif option_type == 'put':
        price = np.exp(-r*T) * (K*norm.cdf(-d2) - (S0*np.exp(param1 +( param2**2)/2) ) * norm.cdf(-d1) )
    else:
        raise ValueError("Option type must be either 'call' or 'put'.")

    return price

In [3]:
def pricer_mc(S0, K, r, sigma, T, n, M, option_type):
    
    dt = T/n
    ranvec = np.random.normal(0, 1,(M, n))

    Spath1 = S0*np.concatenate((np.ones((M,1)),np.cumprod(np.exp((r-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*(ranvec)), axis=1)), axis=1)
    Spath2 = S0*np.concatenate((np.ones((M,1)),np.cumprod(np.exp((r-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*(-ranvec)), axis=1)), axis=1)
    U1 = np.average(Spath1, axis=1)
    print(np.average(U1))
    print(np.average(U1**2))
    U2 = np.average(Spath2, axis=1)
    C = 0.5*np.exp(-r*T)*(np.maximum(U1 - K, 0) + np.maximum(U2 - K, 0) )
    P = 0.5*np.exp(-r*T)*(np.maximum(K - U1, 0) + np.maximum(K - U2, 0) )
    price_c = np.mean(C)
    price_p = np.mean(P)
    stderr_c = np.std(C)/np.sqrt(M)
    stderr_p = np.std(P)/np.sqrt(M)
    confint_c = [price_c - 1.96*stderr_c, price_c + 1.96*stderr_c]
    confint_p = [price_p - 1.96*stderr_p, price_p + 1.96*stderr_p]
    if option_type == 'call':
        price = price_c
        confint = confint_c
    elif option_type == 'put':
        price = price_p
        confint = confint_p
    else:
        raise ValueError("Option type must be either 'call' or 'put'.")

    return [price] + confint

In [42]:
S0 = 100
K = 100
r = 0.05
sigma = 0.3
T = 1
n = 100
option_type = "call"
M = 10000

In [43]:
pricer_mm(S0, K, r, sigma, T, n, option_type)

102.5678304365318
10851.997789534893
[0.009826259104564049, 0.17622653021258566]
0.951229424500714
0.6069199774755943
52.22332007867472


8.055655385255257

In [44]:
pricer_mc(S0, K, r, sigma, T, n, M, option_type)

102.53240408687974
10840.25141976761


[7.940420186808458, 7.815863708681376, 8.06497666493554]

In [ ]:
np.random.normal(0, 1,(M, n))

In [ ]:
np.exp((r-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*(ranvec))

In [ ]:
np.cumprod(np.exp((r-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*(ranvec)), axis=1)

## Barrier options pricing

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

current = Path().resolve()
sys.path.append(str(current))
sys.path.append(str(current.parents[1]))

from src.enums import *
from src.utils import *
from src.market_data import *
from src.pricer import *
# Make charts interactive
%matplotlib notebook

# Initialize market data
MarketData.initialize()

In [13]:
'''
Vanilla Call option price
Inputs:
 k :strike
 r: interest rate
 x: underlying price at time 0
 tau: time to expiry
 sigma: volatility
    
'''
def price_european_call(K, r, S0, tau, sigma):
    d1 = (np.log(S0/K) + (r+0.5*(sigma**2))*tau) / (sigma*np.sqrt(tau))
    d2 = (np.log(S0/K) + (r-0.5*(sigma**2))*tau) / (sigma*np.sqrt(tau))
    return S0*norm.cdf(d1)-K*np.exp(-r*tau)*norm.cdf(d2)

'''
Down-and-In Single Lower Barrier Call option price from call option
Additional inputs:
  B: (lower) barrier level at t=0
'''
def price_barrier_down_in_call(K, r, S0, tau, sigma, B):
    misc = r-(sigma**2)/2
    return (B/S0)**(2*misc/(sigma**2))*price_european_call(K,r,(B**2)/S0,tau,sigma)

'''
Down-and-Out Single Lower Barrier Call option price from call option
'''
def price_barrier_down_out_call(K, r, S0, tau, sigma, B):
    return price_european_call(K, r, S0, tau, sigma) - price_barrier_down_in_call(K, r, S0, tau, sigma, B)

'''
Simulate Geometric Brownian motion several times
Inputs:
  S0: underlying price at time 0
  r: return
  sigma: volatility
  T: time to expiry
  n: path length (number of monitoring points)
  M: number of paths
Output: matrix, rows contain the MC paths 
'''
def simulate_GBM(S0, r, sigma, T, n, M):
    dt = T/n
    ranvec = np.random.normal(0, 1,(M, n))
    return S0*np.concatenate((np.ones((M,1)),np.cumprod(np.exp((r-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*(ranvec)), axis=1)), axis=1)

# '''
# Down-and-out call barrier pricer with normal Monte Carlo
# '''
# def price_barrier_down_in_mc(K, r, S0, sigma, B, T, n, M):
#   simi = simulate_GBM(S0, r, sigma, T, n, M)
#   pat=c(1:m)
#   rem = sapply(pat, function(i){
#     ok=((min(sim[ ,i])>L) & (sim[nrow(sim),i]>k))
#     return(ok) })
#   sum(sim[nrow(sim),rem]-k)/m*exp(-r*tt)



In [6]:
list(range(0,10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [11]:
K = 100
r = 0.05
S0 = 100
T = 1
tau = 1
sigma = 0.3
B = 90
# n = 5
# M = 2

In [14]:
print(price_european_call(K, r, S0, tau, sigma))
print(price_barrier_down_in_call(K, r, S0, tau, sigma, B))
print(price_barrier_down_out_call(K, r, S0, tau, sigma, B))
# print(simulate_GBM(S0, r, sigma, T, n, M))

14.231254785985826
4.8384794790555485
9.392775306930277


In [ ]:
simi = simulate_GBM(S0, r, sigma, T, n, M)
print(simi)
min(simulate_GBM(S0, r, sigma, T, n, M))

In [ ]:
np.min(simi, 1)